# Image classification on CIFAR with a topographic ResNet18

The following notebook trains a topographic ResNet18 from scratch on CIFAR-10
and achieves 0.92 test accuracy after 100 epochs with the given
hyperparameters.

## Imports

In [ ]:
import random

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms

from topography import TopographicLoss, TopographicModel
from topography.models import resnet18
from topography.training import Writer, evaluate, train
from topography.utils import LinearWarmupCosineAnnealingLR

## Hyperparameters and random seed

In [ ]:
seed = 0  # Random seed
root = "./cifar10_topo"  # Output directory
num_classes = 10  # Number of CIFAR classes. Must be 10 or 100
epochs = 100  # Number of training epochs
batch_size = 256  # Batch size
lr = 0.01  # Base learning rate
weight_decay = 0.01  # Weight decay
momentum = 0.9  # SGD momentum
val_proportion = 0.1  # Proportion of the full train set for the validation set

lambd = 0.1
dimension = 2
norm = "euclidean"

In [ ]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Data loading

In [ ]:
train_transform = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
        ),
    ]
)

test_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(
            (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
        ),
    ]
)

dataset = (
    torchvision.datasets.CIFAR10
    if num_classes == 10
    else torchvision.datasets.CIFAR100
)

train_set = dataset(
    root="../../data", train=True, download=True, transform=train_transform
)

val_set = dataset(
    root="../../data", train=True, download=True, transform=test_transform
)

num_train = len(train_set)
indices = torch.randperm(num_train)
split = int(np.floor(val_proportion * num_train))
train_idx, val_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    sampler=train_sampler,
    num_workers=2,
    pin_memory=True,
)

val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=batch_size,
    sampler=val_sampler,
    num_workers=2,
    pin_memory=True,
)

test_set = dataset(
    root="../../data", train=False, download=True, transform=test_transform
)
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
)

## Defining the main componenents

In [ ]:
from topography.core.loss import MetricOutput

device = "cuda" if torch.cuda.is_available() else "cpu"
cross_entropy = nn.CrossEntropyLoss()
topo_loss = TopographicLoss()

model = TopographicModel(
    resnet18(num_classes=num_classes), dimension=dimension, norm=norm
).to(device)
optimizer = optim.SGD(
    model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay
)
scheduler = LinearWarmupCosineAnnealingLR(
    optimizer, warmup_epochs=epochs * 0.3, max_epochs=epochs
)
writer = Writer(f"{root}/runs")


def criterion(output, target):
    ce = cross_entropy(output, target)
    topo = topo_loss(model.activations, model.inverse_distance)
    return MetricOutput(
        value=ce + lambd * topo.value,
        extras={
            "loss-cross-entropy": ce.item(),
            "loss-topographic": topo.value.item(),
            **topo.extras,
        },
    )


writer.log_config(
    dict(
        num_classes=num_classes,
        val_proportion=val_proportion,
        epochs=epochs,
        batch_size=batch_size,
        lr=lr,
        lambd=lambd,
        weight_decay=weight_decay,
        momentum=momentum,
        optimizer="sgd",
        scheduler="LinearWarmupCosineAnnealingLR",
    )
)

## Training the model

In [ ]:
for _ in range(epochs):
    train(model, train_loader, optimizer, criterion, device, writer)
    evaluate(model, test_loader, criterion, device, writer, mode="val")
    scheduler.step()
    writer.save(
        "val", "acc", model=model, optimizer=optimizer, scheduler=scheduler
    )

## Final evaluation

In [ ]:
evaluate(model, test_loader, criterion, device, writer, mode="test")
writer.close()